# OLG Model Transition Path Analysis
移行過程分析と結果の可視化

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from olg_solver import solve_ss, Setting, SteadyStateResult, TransitionSetting
from olg_transition_solver import create_capital_guess, solve_transition_path

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. OLG Model Setup and Transition Calculation
移行過程の設定と計算

In [7]:
def create_policy_function_boxes(tr_setting, setting):
    """
    移行過程の政策関数とそのインデックスの箱を作成
    - opt_indexes: 政策関数インデックス (NT, NJ, Nl, Na)
    - aprimes: 政策関数実数値 (NT, NJ, Nl, Na)
    """
    shape = (tr_setting.NT, setting.NJ, setting.Nl, setting.Na)
    opt_indexes = np.zeros(shape, dtype=np.int32)
    aprimes = np.zeros(shape, dtype=np.float64)
    return opt_indexes, aprimes

def run_transition_analysis():
    """移行過程分析を実行"""
    print("=== OLG移行過程分析 ===")
    
    # 移行過程の設定を作成
    tr_setting = TransitionSetting(
        NT=50,    # 移行期間
        TT=25,     # 政策変更期間
        psi_ini=0.5,  # 初期所得代替率
        psi_fin=0.25  # 最終所得代替率
    )
    
    tr_setting.print_transition_summary()
    
    # 初期・最終定常状態用のSetting作成
    print("\n=== 定常状態用設定の作成 ===")
    initial_setting, final_setting = tr_setting.create_ss_settings(
        Na=101,        # 資産グリッド数
        Naprime=1001   # 政策関数用資産グリッド数
    )
    
    print(f"初期定常状態設定: ψ = {initial_setting.psi:.3f}")
    print(f"最終定常状態設定: ψ = {final_setting.psi:.3f}")
    
    # 1. 初期・最終定常状態の計算
    print("\n=== 初期定常状態の計算 ===")
    initial_result = solve_ss(initial_setting)
    mu_ini = initial_result.mu_dist_box
    K_ini = initial_result.K
    
    print("\n=== 最終定常状態の計算 ===")
    final_result = solve_ss(final_setting)
    V_fin = final_result.value_fun_box
    K_fin = final_result.K
    
    # 2. 移行過程の初期設定
    print("\n=== 移行過程の初期設定 ===")
    K_path = create_capital_guess(tr_setting, K_ini, K_fin)
    
    # 政策関数とそのインデックスの箱を用意
    opt_indexes, aprimes = create_policy_function_boxes(tr_setting, initial_setting)

    # 移行過程の反復計算
    print("\n=== 移行過程の反復計算 ===")
    converged_K_path = solve_transition_path(
        tr_setting, initial_setting, K_path, opt_indexes, aprimes, V_fin, mu_ini
    )
    
    return tr_setting, initial_result, final_result, converged_K_path, opt_indexes, aprimes

In [ ]:
# 移行過程分析を実行
tr_setting, initial_result, final_result, K_path, opt_indexes, aprimes = run_transition_analysis()

=== OLG移行過程分析 ===

=== 移行過程設定サマリー ===
移行期間: 50 期
政策変更収束期間: 25 期
初期所得代替率: 0.500
最終所得代替率: 0.250
移行過程用収束判定閾値: 1.0e-04
移行過程用最大繰り返し回数: 300
移行過程用資本更新調整係数: 0.050

主要期間の所得代替率:
  第1期: ψ = 0.500, τ = 0.178
  第7期: ψ = 0.438, τ = 0.156
  第13期: ψ = 0.375, τ = 0.133
  第25期: ψ = 0.250, τ = 0.089
  第50期: ψ = 0.250, τ = 0.089

=== 定常状態用設定の作成 ===
初期定常状態設定: ψ = 0.500
最終定常状態設定: ψ = 0.250

=== 初期定常状態の計算 ===
numba最適化されたOLGモデルを実行中...
Iteration 1: market_diff = 6.755121e-01, errm = 2.220446e-16
Iteration 2: market_diff = 1.254751e-01, errm = 2.220446e-16
Iteration 3: market_diff = 2.663168e-02, errm = 0.000000e+00
Iteration 4: market_diff = 9.566629e-03, errm = 0.000000e+00
Iteration 5: market_diff = 3.647950e-03, errm = 0.000000e+00
\n計算完了! 実行時間: 1.88秒
総イテレーション数: 8
最終市場差: 8.982731e-05
最終人口合計誤差: 0.000000e+00
収束した資本ストック K = 6.1685

=== 最終定常状態の計算 ===
numba最適化されたOLGモデルを実行中...
Iteration 1: market_diff = 3.168984e+00, errm = 0.000000e+00
Iteration 2: market_diff = 7.061519e-01, errm = 0.000000e+00
Iteration 3: mar

## 2. Capital Path Analysis
資本パスの分析と可視化

In [ ]:
def plot_capital_path(tr_setting, K_path, initial_result, final_result):
    """資本パスをプロット"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 資本パス
    periods = np.arange(1, tr_setting.NT + 1)
    ax1.plot(periods, K_path, 'b-', linewidth=2, label='Capital Path')
    ax1.axhline(y=initial_result.K, color='red', linestyle='--', alpha=0.7, label=f'Initial SS (K={initial_result.K:.3f})')
    ax1.axhline(y=final_result.K, color='green', linestyle='--', alpha=0.7, label=f'Final SS (K={final_result.K:.3f})')
    ax1.set_xlabel('Period')
    ax1.set_ylabel('Capital Stock')
    ax1.set_title('Capital Stock Transition Path')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 政策パラメータ（ψ）の変化
    psi_path = tr_setting.rhoT
    ax2.plot(periods, psi_path, 'r-', linewidth=2, label='Income Replacement Rate (ψ)')
    ax2.set_xlabel('Period')
    ax2.set_ylabel('ψ (Income Replacement Rate)')
    ax2.set_title('Policy Parameter Transition')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_capital_path(tr_setting, K_path, initial_result, final_result)

## 3. Factor Prices Analysis
要素価格の分析

In [ ]:
def calculate_factor_prices_path(tr_setting, K_path, initial_setting):
    """移行過程の要素価格を計算"""
    L = initial_setting.Njw / initial_setting.NJ  # 労働供給
    
    r_path = np.zeros(tr_setting.NT)
    w_path = np.zeros(tr_setting.NT)
    
    for t in range(tr_setting.NT):
        K_t = K_path[t]
        r_path[t] = initial_setting.alpha * (K_t / L) ** (initial_setting.alpha - 1) - initial_setting.delta
        w_path[t] = (1 - initial_setting.alpha) * (K_t / L) ** initial_setting.alpha
    
    return r_path, w_path

def plot_factor_prices(tr_setting, K_path, initial_setting, initial_result, final_result):
    """要素価格をプロット"""
    r_path, w_path = calculate_factor_prices_path(tr_setting, K_path, initial_setting)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    periods = np.arange(1, tr_setting.NT + 1)
    
    # 利子率
    ax1.plot(periods, r_path, 'b-', linewidth=2, label='Interest Rate')
    ax1.axhline(y=initial_result.r, color='red', linestyle='--', alpha=0.7, label=f'Initial SS (r={initial_result.r:.4f})')
    ax1.axhline(y=final_result.r, color='green', linestyle='--', alpha=0.7, label=f'Final SS (r={final_result.r:.4f})')
    ax1.set_xlabel('Period')
    ax1.set_ylabel('Interest Rate')
    ax1.set_title('Interest Rate Transition')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 賃金率
    ax2.plot(periods, w_path, 'g-', linewidth=2, label='Wage Rate')
    ax2.axhline(y=initial_result.w, color='red', linestyle='--', alpha=0.7, label=f'Initial SS (w={initial_result.w:.3f})')
    ax2.axhline(y=final_result.w, color='green', linestyle='--', alpha=0.7, label=f'Final SS (w={final_result.w:.3f})')
    ax2.set_xlabel('Period')
    ax2.set_ylabel('Wage Rate')
    ax2.set_title('Wage Rate Transition')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_factor_prices(tr_setting, K_path, initial_result.setting, initial_result, final_result)

## 4. Policy Function Analysis
政策関数の分析

In [ ]:
def plot_policy_functions(tr_setting, aprimes, initial_result, periods_to_show=[0, 4, 9, 24, 49]):
    """特定期間の政策関数をプロット"""
    setting = initial_result.setting
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    # 中年（40歳）、高生産性の政策関数を表示
    age_idx = 20  # 40歳（0ベース）
    skill_idx = 1  # 高生産性
    
    for i, period in enumerate(periods_to_show):
        if i < len(axes):
            ax = axes[i]
            ax.plot(setting.a_grid, aprimes[period, age_idx, skill_idx, :], 'b-', linewidth=2)
            ax.plot(setting.a_grid, setting.a_grid, 'k--', alpha=0.5, label='45° line')
            ax.set_xlabel('Current Assets')
            ax.set_ylabel('Next Period Assets')
            ax.set_title(f'Policy Function: Period {period+1} (Age 40, High Skill)')
            ax.legend()
            ax.grid(True, alpha=0.3)
    
    # 定常状態比較
    if len(axes) > len(periods_to_show):
        ax = axes[len(periods_to_show)]
        ax.plot(setting.a_grid, initial_result.afun_box[age_idx, skill_idx, :], 'r-', linewidth=2, label='Initial SS')
        ax.plot(setting.a_grid, final_result.afun_box[age_idx, skill_idx, :], 'g-', linewidth=2, label='Final SS')
        ax.plot(setting.a_grid, setting.a_grid, 'k--', alpha=0.5, label='45° line')
        ax.set_xlabel('Current Assets')
        ax.set_ylabel('Next Period Assets')
        ax.set_title('Steady State Comparison (Age 40, High Skill)')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_policy_functions(tr_setting, aprimes, initial_result)

## 5. Age Profile Analysis
年齢プロファイルの分析

In [ ]:
def plot_age_profiles(tr_setting, aprimes, initial_result, periods_to_show=[0, 24, 49]):
    """年齢プロファイルをプロット"""
    setting = initial_result.setting
    ages = np.arange(20, 20 + setting.NJ)  # 20歳から開始と仮定
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 資産中位値（a_grid の中央値を使用）での次期資産選択
    median_asset_idx = len(setting.a_grid) // 2
    skill_idx = 1  # 高生産性
    
    # 移行過程での年齢プロファイル
    for period in periods_to_show:
        profile = aprimes[period, :, skill_idx, median_asset_idx]
        ax1.plot(ages, profile, linewidth=2, label=f'Period {period+1}')
    
    ax1.set_xlabel('Age')
    ax1.set_ylabel('Next Period Assets')
    ax1.set_title('Age Profile of Asset Choice (High Skill, Median Assets)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 定常状態比較
    initial_profile = initial_result.afun_box[:, skill_idx, median_asset_idx]
    final_profile = final_result.afun_box[:, skill_idx, median_asset_idx]
    
    ax2.plot(ages, initial_profile, 'r-', linewidth=2, label='Initial SS')
    ax2.plot(ages, final_profile, 'g-', linewidth=2, label='Final SS')
    ax2.set_xlabel('Age')
    ax2.set_ylabel('Next Period Assets')
    ax2.set_title('Steady State Age Profiles (High Skill, Median Assets)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_age_profiles(tr_setting, aprimes, initial_result)

## 6. Aggregate Statistics
集計統計の分析

In [ ]:
def calculate_aggregate_stats(tr_setting, K_path, initial_setting, initial_result, final_result):
    """集計統計を計算"""
    r_path, w_path = calculate_factor_prices_path(tr_setting, K_path, initial_setting)
    
    # GDP per capita (Y/L)
    L = initial_setting.Njw / initial_setting.NJ
    Y_path = np.zeros(tr_setting.NT)
    for t in range(tr_setting.NT):
        Y_path[t] = (K_path[t] / L) ** initial_setting.alpha
    
    # Capital-output ratio
    KY_ratio = K_path / (Y_path * L)
    
    return {
        'periods': np.arange(1, tr_setting.NT + 1),
        'capital': K_path,
        'interest_rate': r_path,
        'wage_rate': w_path,
        'output_per_capita': Y_path,
        'capital_output_ratio': KY_ratio,
        'initial_ss': {
            'K': initial_result.K,
            'r': initial_result.r,
            'w': initial_result.w
        },
        'final_ss': {
            'K': final_result.K,
            'r': final_result.r,
            'w': final_result.w
        }
    }

def plot_aggregate_dashboard(stats):
    """集計統計のダッシュボード"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Capital stock
    axes[0,0].plot(stats['periods'], stats['capital'], 'b-', linewidth=2)
    axes[0,0].axhline(y=stats['initial_ss']['K'], color='red', linestyle='--', alpha=0.7)
    axes[0,0].axhline(y=stats['final_ss']['K'], color='green', linestyle='--', alpha=0.7)
    axes[0,0].set_title('Capital Stock')
    axes[0,0].set_xlabel('Period')
    axes[0,0].grid(True, alpha=0.3)
    
    # Interest rate
    axes[0,1].plot(stats['periods'], stats['interest_rate'], 'r-', linewidth=2)
    axes[0,1].axhline(y=stats['initial_ss']['r'], color='red', linestyle='--', alpha=0.7)
    axes[0,1].axhline(y=stats['final_ss']['r'], color='green', linestyle='--', alpha=0.7)
    axes[0,1].set_title('Interest Rate')
    axes[0,1].set_xlabel('Period')
    axes[0,1].grid(True, alpha=0.3)
    
    # Output per capita
    axes[1,0].plot(stats['periods'], stats['output_per_capita'], 'g-', linewidth=2)
    axes[1,0].set_title('Output per Capita')
    axes[1,0].set_xlabel('Period')
    axes[1,0].grid(True, alpha=0.3)
    
    # Capital-output ratio
    axes[1,1].plot(stats['periods'], stats['capital_output_ratio'], 'purple', linewidth=2)
    axes[1,1].set_title('Capital-Output Ratio')
    axes[1,1].set_xlabel('Period')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 集計統計を計算してプロット
stats = calculate_aggregate_stats(tr_setting, K_path, initial_result.setting, initial_result, final_result)
plot_aggregate_dashboard(stats)

## 7. Summary Statistics
要約統計

In [ ]:
def print_summary_stats(tr_setting, stats):
    """要約統計を表示"""
    print("=== TRANSITION ANALYSIS SUMMARY ===")
    print(f"Transition periods: {tr_setting.NT}")
    print(f"Policy change periods: {tr_setting.TT}")
    print(f"Income replacement rate: {tr_setting.psi_ini:.3f} → {tr_setting.psi_fin:.3f}")
    print()
    
    print("=== STEADY STATE COMPARISON ===")
    print(f"Capital Stock:   {stats['initial_ss']['K']:.4f} → {stats['final_ss']['K']:.4f} ({(stats['final_ss']['K']/stats['initial_ss']['K']-1)*100:+.2f}%)")
    print(f"Interest Rate:   {stats['initial_ss']['r']:.4f} → {stats['final_ss']['r']:.4f} ({(stats['final_ss']['r']-stats['initial_ss']['r'])*10000:+.2f} bp)")
    print(f"Wage Rate:       {stats['initial_ss']['w']:.4f} → {stats['final_ss']['w']:.4f} ({(stats['final_ss']['w']/stats['initial_ss']['w']-1)*100:+.2f}%)")
    print()
    
    print("=== TRANSITION DYNAMICS ===")
    # 収束までの期間を推定
    K_change = np.abs(stats['capital'] - stats['final_ss']['K'])
    convergence_threshold = 0.01 * abs(stats['final_ss']['K'] - stats['initial_ss']['K'])
    convergence_period = np.where(K_change < convergence_threshold)[0]
    
    if len(convergence_period) > 0:
        print(f"Approximate convergence: Period {convergence_period[0]+1}")
    else:
        print("Convergence: Not achieved within transition period")
    
    print(f"Maximum capital: {np.max(stats['capital']):.4f} (Period {np.argmax(stats['capital'])+1})")
    print(f"Minimum capital: {np.min(stats['capital']):.4f} (Period {np.argmin(stats['capital'])+1})")
    
print_summary_stats(tr_setting, stats)

## 8. Comparative Analysis (Optional)
比較分析（異なるパラメータでの実験）

In [ ]:
def compare_different_policies():
    """異なる政策変更幅での比較分析"""
    policy_scenarios = [
        {'psi_ini': 0.5, 'psi_fin': 0.4, 'label': 'Small reduction (0.5→0.4)'},
        {'psi_ini': 0.5, 'psi_fin': 0.25, 'label': 'Large reduction (0.5→0.25)'},
        {'psi_ini': 0.5, 'psi_fin': 0.1, 'label': 'Very large reduction (0.5→0.1)'}
    ]
    
    plt.figure(figsize=(12, 8))
    
    for scenario in policy_scenarios:
        # 簡易版の移行過程計算（実際の計算は時間がかかるため、ここではスキップ）
        # 実際に実行する場合は、各シナリオで run_transition_analysis() を呼び出す
        print(f"Scenario: {scenario['label']}")
        print(f"  Policy change: ψ = {scenario['psi_ini']} → {scenario['psi_fin']}")
        print(f"  Impact: {(scenario['psi_fin']/scenario['psi_ini']-1)*100:+.1f}% change in replacement rate")
        print()
    
    print("Note: To run full comparative analysis, uncomment and modify the code below:")
    print("# for scenario in policy_scenarios:")
    print("#     tr_setting_comp = TransitionSetting(NT=50, TT=5, psi_ini=scenario['psi_ini'], psi_fin=scenario['psi_fin'])")
    print("#     # Run analysis and plot results...")

compare_different_policies()